# Journal Entries Analysis
A mini data analysis project to examine and analyze the contents of journal entries I have written from 2019 to 2025.

In [1]:
import json
import psycopg2
import glob
from datetime import datetime

### Inserting data into Postgres DB
Schema was created beforehand on psql

In [8]:
DB_PARAMS = {
    "dbname": "journal_db", 
    "user": "postgres",
    "password": "password",
    "host": "localhost"
}

In [ ]:
# Connect to postgres db
conn = psycopg2.connect(**DB_PARAMS)
cursor = conn.cursor()

In [ ]:
# Load JSON files into DB
for file in glob.glob("data/*.json"):  
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

        # Convert milliseconds timestamp to PostgreSQL TIMESTAMP format
        date_journal = datetime.utcfromtimestamp(data["date_journal"] / 1000)

        # Insert into PostgreSQL
        cursor.execute("""
            INSERT INTO journal_entries (id, date_journal, text, tags, mood, sentiment, address, latitude, longitude, weather_description, weather_temp, weather_place)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            data["id"], 
            date_journal, 
            data["text"], 
            data["tags"], 
            data["mood"], 
            data["sentiment"], 
            data["address"], 
            data["lat"], 
            data["lon"], 
            data["weather"]["description"], 
            data["weather"]["degree_c"], 
            data["weather"]["place"]
        ))

In [7]:
# Commit and close connection
conn.commit()
cursor.close()
conn.close()

---

### Initial Analysis

In [9]:
conn = psycopg2.connect(**DB_PARAMS)